In [1]:
import pandas as pd


In [4]:
df = pd.read_csv('../data/parsed_logs_rad_may_2024.csv')

In [12]:
grouped_df = df.groupby(["hostname",  "ecc", "model", "floatthreshold", "microop", "setup_type", "hardenedid"]).agg({"batchsize": max, "acc_time": sum, "sdc": sum, "critical_sdc": sum})

In [13]:
df

,testsamples,batchsize,model,setup_type,floatthreshold,loghelperinterval,precision,microop,hardenedid,lognvml,...,ecc,hostname,logfile,it,ker_time,acc_time,ker_err,acc_err,sdc,critical_sdc
0,32,8,vit_base_patch16_224,vits,0.01,10,fp32,None,True,False,...,OFF,carola20001,2024_05_23_02_06_41_vit_base_patch16_224_ECC_O...,1006,0.097372,97.179886,1,1,1,0
1,32,8,vit_base_patch16_224,vits,0.01,10,fp32,None,True,False,...,OFF,carola20001,2024_05_23_05_55_46_vit_base_patch16_224_ECC_O...,231,0.093556,0.000000,1,1,1,0
2,32,8,vit_base_patch16_224,vits,0.01,10,fp32,None,True,False,...,OFF,carola20001,2024_05_23_05_55_46_vit_base_patch16_224_ECC_O...,685,0.094413,64.754208,1,2,1,0
3,32,8,vit_base_patch16_224,vits,0.10,10,fp16,None,False,False,...,OFF,carola20001,2024_05_22_21_28_28_vit_base_patch16_224_ECC_O...,616,0.017988,11.296170,2,2,1,1
4,32,8,vit_base_patch16_224,vits,0.01,10,fp32,None,False,False,...,OFF,carola20001,2024_05_23_04_48_52_vit_base_patch16_224_ECC_O...,1220,0.078732,95.650315,8,8,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730,8,1,groundingdino_swinb_cogcoor,grounding_dino,0.01,1,fp32,Attention,False,False,...,OFF,carolp20002,2024_05_23_00_34_05_groundingdino_swinb_cogcoo...,12,1.223616,0.000000,3,15,1,0
1731,8,1,groundingdino_swinb_cogcoor,grounding_dino,0.01,1,fp32,Attention,False,False,...,OFF,carolp20002,2024_05_23_00_34_05_groundingdino_swinb_cogcoo...,14,1.387503,0.000000,3,18,1,0
1732,8,1,groundingdino_swinb_cogcoor,grounding_dino,0.01,1,fp32,Attention,False,False,...,OFF,carolp20002,2024_05_23_00_34_05_groundingdino_swinb_cogcoo...,15,1.373685,19.135720,3,21,1,0
1733,8,1,groundingdino_swint_ogc,grounding_dino,0.01,1,fp32,Attention,True,False,...,OFF,carolp20002,2024_05_23_03_24_25_groundingdino_swint_ogc_EC...,1,1.149857,0.000000,3,3,1,0


In [15]:
grouped_df["err_rate"] = grouped_df["sdc"] / grouped_df["acc_time"]

In [16]:
grouped_df

batchsize  \
hostname    ecc model                       floatthreshold microop   setup_type     hardenedid              
carola20001 OFF vit_base_patch16_224        0.01           None      vits           False               8   
                                                                                    True                8   
                                            0.10           None      vits           False               8   
                                                                                    True                8   
                vit_base_patch16_384        0.01           Attention microbenchmark False               8   
                                                           Block     microbenchmark False               8   
carolp20002 OFF groundingdino_swinb_cogcoor 0.01           Attention grounding_dino False               1   
                                                                                    True                1   
                                            0.10           Attention grounding_dino False               1   
                                                                                    True                1   
                groundingdino_swint_ogc     0.01           Attention grounding_dino False               1   
                                                                                    True                1   

                                                                                                   acc_time  \
hostname    ecc model                       floatthreshold microop   setup_type     hardenedid                
carola20001 OFF vit_base_patch16_224        0.01           None      vits           False       1899.688553   
                                                                                    True        1463.919021   
                                            0.10           None      vits           False       1669.635372   
                                                                                    True         196.248820   
                vit_base_patch16_384        0.01           Attention microbenchmark False       1195.039556   
                                                           Block     microbenchmark False       2698.228651   
carolp20002 OFF groundingdino_swinb_cogcoor 0.01           Attention grounding_dino False       3923.360195   
                                                                                    True        1427.972256   
                                            0.10           Attention grounding_dino False       5554.948415   
                                                                                    True         642.790403   
                groundingdino_swint_ogc     0.01           Attention grounding_dino False       5765.238702   
                                                                                    True        2510.804299   

                                                                                                sdc  \
hostname    ecc model                       floatthreshold microop   setup_type     hardenedid        
carola20001 OFF vit_base_patch16_224        0.01           None      vits           False        14   
                                                                                    True        242   
                                            0.10           None      vits           False        24   
                                                                                    True          3   
                vit_base_patch16_384        0.01           Attention microbenchmark False        19   
                                                           Block     microbenchmark False        52   
carolp20002 OFF groundingdino_swinb_cogcoor 0.01           Attention grounding_dino False       422   
                                                                                    Tru